In [1]:
from docx import Document
from pathlib import Path
import pandas as pd

In [2]:
def create_df(table):
    num_row = len(table.rows)
    num_col = len(table.rows[0].cells)
    for i in range(num_row):
        row = table.rows[i]
        if i == 0:
            header = [x.text.strip() for x in row.cells]
            data = {x:[] for x in header}
        else:
            for j in range(num_col):
                key = header[j]
                item = row.cells[j].text.strip()
                data[key].append(item)
    df = pd.DataFrame(data)
    return df

In [3]:
from bs4 import BeautifulSoup
import re
import requests

In [4]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'

In [5]:
def get_url(url):
    headers = {
        'User-Agent': USER_AGENT
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    url = soup.find('img', {'class': 'exImg'})['data-url_male']
    return url

In [6]:
from alive_progress import alive_it, config_handler

In [7]:
config_handler.set_global(
    length=75,
    spinner='classic',
    bar='classic2',
    force_tty=True,
    dual_line=True
)

In [8]:
def get_images(df):
    links = list(df['Link'])
    results = alive_it(links)
    images = []
    for index, result in enumerate(results):
        url = get_url(result)
        images.append(url)
    return images

In [9]:
# def create_json():
file_path = Path.cwd() / 'workout.docx'
document = Document(file_path)
tables = document.tables

#keys = ['day1', 'day2', 'day3', 'day4']
#for table in tables:
    
table = tables[0]
df = create_df(table)
df['Image'] = get_images(df)

output_path = Path.cwd() / 'data.json'
df.to_json(output_path, orient='records')

[###########################################################################] 10/10 [100%] in 18.5s (0.54/s)            
